In [489]:
import re
from bs4 import BeautifulSoup 
#from lxml import etree as LX
from os import walk
from transliterate import translit, get_available_language_codes

In [490]:
def transliterate (somecyrstring):
    #print (somecyrstring)
    transliterated = translit (somecyrstring, 'ru', reversed=True)
    transliterated_clean = re.sub ('[.,!:&? -]','',transliterated)
    #print (transliterated)
    return transliterated_clean

In [491]:
def determine_sex (persname):
    if persname.endswith ('ә'):
        return 'FEMALE'
    else:
        return 'MALE'

In [492]:
def add_perslist_to_header (header, perslist):
    xmlperslist = ''
    for person_id in perslist:
        persname = perslist[person_id]
        sex = determine_sex (persname)
        pl_item = """<person xml:id="{}" sex="{}">
        <persName>{}</persName>
        </person>\n""".format(person_id,sex,persname)
        xmlperslist += pl_item
    xmlperslist = '<listPerson>{}</listPerson>'.format (xmlperslist)
    return re.sub('INSERTPERSLISTHERE',xmlperslist, header)

In [493]:
def add_header_with_meta (string_with_play_content,soup, perslist):
    header = open('header_stub.xml','r').read()
    play_title = soup.title.text
    header = re.sub('INSERTTITLEHERE', play_title, header)
    header_with_cast_list = add_perslist_to_header (header, perslist) 
    return (header_with_cast_list + string_with_play_content) 

In [494]:
def add_to_perslist (perslist, speaker_for_list):
    speaker_for_list_name = re.sub ('[.,!:&? ]','',speaker_for_list[1])
    speaker_id = speaker_for_list[0]
    perslist[speaker_id] = speaker_for_list_name

In [495]:
def html_line_to_tei (item):
    newstring = str (item)
    thislinekids = item.contents
    speaker_for_list = None
    if thislinekids[0].name == 'b':
        newstring = re.sub('<(/)?b>','<\\1speaker>', newstring)
        nameforid = transliterate (thislinekids[0].text)
        speaker_for_list = (nameforid,thislinekids[0].text)
        #print (speaker_for_list)
        newstring = re.sub('<p>','<sp who="'+nameforid+'">', newstring)
        newstring = re.sub('</p>','</sp>', newstring)
        newstring = re.sub('<(/)?i>','<\\1stage>', newstring)
    elif thislinekids[0].name == 'i':
        newstring = re.sub('<(/)?p>','', newstring)
        newstring = re.sub('<(/)?i>','<\\1stage>', newstring)
    newstring = re.sub ('</speaker>(.*)</sp>','</speaker><p>\\1</p></sp>',newstring)
    return (newstring, speaker_for_list)
    #thislinekids = item.contents
    #thislinekids[]
    
    #parse_line_plain(item)
        #print ('speaker: ',thislinekids[0].text)
        #print ('speech: ', item.text)
    

In [496]:
def move_html_to_xml (soup):
    #teibody=LX.Element('body') # создаем body нашего нового TEI
    string_with_play_content = ''
    perslist = {}
    play_content = soup.find ('div', id="content")
    for item in play_content:
        #print (item.name)
        speaker_for_list = None
        if item.name == 'p':
            if item.children[0].name == 'b'
                if re.search ('нч(е|ы) манзара',item.children[0].text,re.IGNORECASE):
                    print (item.children[0].text)
                    line_to_write = '</div>\n<div type="act"><head>{}</head>'.format(item.text)
                elif re.search ('нч(е|ы) мәҗлес',item.children[0].text,re.IGNORECASE):
                    print (item.children[0].text)
                    line_to_write = '</div>\n<div type="scene"><head>{}</head>'.format(item.text)
            else:
                line_to_write, speaker_for_list = html_line_to_tei (item)
        #elif item.name == 'br':
         #   line_to_write = '</div>\n<div>'#html_break_to_tei (item)
        else:
            line_to_write = None
        if line_to_write != None:
            string_with_play_content += line_to_write + '\n'
        if speaker_for_list != None:
            add_to_perslist (perslist, speaker_for_list)
    #print (perslist)
    body_with_content = '<text>\n<div>\n{}\n</div>\n</text>'.format (string_with_play_content)
    body_with_content_and_meta = add_header_with_meta (body_with_content,soup , perslist)
    ultimate_xml = '<TEI>\n{}\n</TEI>'.format (body_with_content_and_meta)
    return (ultimate_xml)

In [497]:
def parse_html(inputfile):
    soup = BeautifulSoup(inputfile, 
                         'html.parser', 
                         from_encoding ='cp1251')
    #xml_tree = LXET.ElementTree
    print (soup.title.string)
    #print (soup.prettify())
    return (move_html_to_xml (soup))

In [498]:
def out_write(out_string, filename):
    newname = re.sub ('\.html?','.xml', filename)
    newpath = '../Parsed/'
    outfile = open (newpath + newname, 'w')
    outfile.write (out_string)
    outfile.close()

In [499]:
def parsefolder (folderwithhtml):#, header):
    for path, dirs, filenames in walk (folderwithhtml):
        for filename in filenames:
            if '.htm' in filename:
                print (filename)
                openhtml = open(path+'/'+filename, 
                                'r', 
                                encoding="cp1251")
                #parse_html(openhtml)
                out_string = parse_html(openhtml)
                out_write (out_string, filename)
                openhtml.close()

In [500]:
folderwithhtml = '../DramaSources/ToParse'
parsefolder (folderwithhtml)

qamal-kaynish.htm
Татарская электронная библиотека: Галиаскар Камал. Кайниш
Пьеса 1 пәрдәдә 2 манзарада (картинада)
БЕРЕНЧЕ МАНЗАРА
Беренче мәҗлес
Икенче мәҗлес
Өченче мәҗлес
Дүртенче мәҗлес
Бишенче мәҗлес
Алтынчы мәҗлес
Җиденче мәҗлес
ИКЕНЧЕ МАНЗАРА
Беренче мәҗлес ,
Икенче мәҗлес
 Өченче мәҗлес
Дүртенче мәҗлес
qamal-beznen-shehernen-serlere.htm
Татарская электронная библиотека: Галиаскар Камал. Безнең шәһәрнең серләре
Манзара биш пәрдәдә
БЕРЕНЧЕ МАНЗАРА
Беренче мәҗлес
Сакчы. «Ул нәрсә ул тагы?» — ди. Идарә мәҗлесенә записка, дигән идем, ачуланырга тотынды. «Бетмәде инде мәҗлесегез, — ди. — Көн дә, көн дә нинди мәҗлес булсын, бүген бармас, бүген безнең парлы кунакка барасыбыз бар. һәр көн кич анда барып, төн уздырып кайтырга булмас», — ди.
Сакчы. Касыйм абзыйны күргән идем. «Ярар, бушый алсам, барырмын. Минем, ясигъдән чыккач, бер җиргә туйга барасым бар. Миңа карамасыннар, мәҗлесне ачып эшкә башлый торсыннар. Мин пртакулга кул куярга барып җитәрмен. Бара алмасам, иртәгә пртакул дәфтәр